# 18_v5_empty 요약

지피티가 준 코드. MLP 특화 + Optuna


In [1]:
import os, re, json, time, random, warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_auc_score

import optuna


# ============================================================
# CONFIG
# ============================================================
TRAIN_PATH = "../../data/raw/train.csv"
TEST_PATH  = "../../data/raw/test_x.csv"

SEED = 42
N_FOLDS = 5
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

# Optuna
N_TRIALS = 50                 # 시간 되면 200~500까지 늘려도 됨
TIMEOUT_SEC = None            # 예: 60*60*3 (3시간) 등
OPTUNA_STUDY_NAME = "MLP_TE"
OPTUNA_STORAGE = None         # "sqlite:///optuna_mlp.db" 로 하면 중단/재개 가능

# Train
EPOCHS = 80
PATIENCE = 10
BATCH_SIZE = 512

print(f"🖥️ Device: {DEVICE}")
print(f"📂 Train path: {TRAIN_PATH}")
print(f"📂 Test  path: {TEST_PATH}")


def set_seed(seed=SEED):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)


# ============================================================
# Data Load
# ============================================================
train_raw = pd.read_csv(TRAIN_PATH)
test_raw  = pd.read_csv(TEST_PATH)
train_raw["voted_bin"] = (train_raw["voted"] == 2).astype(int)

print(f"Train: {train_raw.shape}, Test: {test_raw.shape}")


# ============================================================
# Preprocess (너가 쓰던 방식 유지)
# ============================================================
def clean_data(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()

    # 무응답 (0 → NaN)
    for col in ["education", "engnat", "hand", "married", "urban"]:
        if col in df.columns:
            df.loc[df[col] == 0, col] = np.nan

    # familysize
    if "familysize" in df.columns:
        df.loc[df["familysize"] == 0, "familysize"] = np.nan
        df.loc[df["familysize"] > 15, "familysize"] = np.nan

    # TP 0 → NaN
    for col in [f"tp{i:02d}" for i in range(1, 11)]:
        if col in df.columns:
            df.loc[df[col] == 0, col] = np.nan

    # Q_E clipping
    for col in [f"Q{c}E" for c in "abcdefghijklmnopqrst"]:
        if col in df.columns:
            df[col] = df[col].clip(lower=100, upper=60000)

    return df


# ============================================================
# Feature Engineering
# - RAW:   원본 문항들(qa/qe/tp/wr/wf)
# - SUM:   요약통계 + 간단 interaction
# - TE:    target encoding (CV 내부에서만 생성)
# ============================================================
def build_features(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()

    # demographic
    age_map = {"10s": 1, "20s": 2, "30s": 3, "40s": 4, "50s": 5, "60s": 6, "+70s": 7}
    df["age_ord"] = df["age_group"].map(age_map)

    df["is_teenager"] = (df["age_ord"] == 1).astype(float)
    df["is_young"] = (df["age_ord"] <= 2).astype(float)
    df["is_old"] = (df["age_ord"] >= 6).astype(float)

    df["edu_low"] = (df["education"] <= 2).astype(float)
    df["edu_high"] = (df["education"] >= 3).astype(float)

    df["is_single"] = (df["married"] == 1).astype(float)
    df["is_married"] = (df["married"] == 2).astype(float)

    df["is_urban"] = (df["urban"] == 3).astype(float)
    df["is_english_native"] = (df["engnat"] == 1).astype(float)
    df["is_male"] = (df["gender"] == "Male").astype(float)

    # Q_A summary
    qa_cols = [f"Q{c}A" for c in "abcdefghijklmnopqrst"]
    df["qa_mean"] = df[qa_cols].mean(axis=1)
    df["qa_std"] = df[qa_cols].std(axis=1)
    df["qa_range"] = df[qa_cols].max(axis=1) - df[qa_cols].min(axis=1)
    df["qa_extreme_ratio"] = ((df[qa_cols] == 1) | (df[qa_cols] == 5)).sum(axis=1) / len(qa_cols)
    df["qa_neutral_ratio"] = (df[qa_cols] == 3).sum(axis=1) / len(qa_cols)
    df["qa_all_same"] = (df[qa_cols].std(axis=1) == 0).astype(float)

    # Q_E log + summary
    qe_cols = [f"Q{c}E" for c in "abcdefghijklmnopqrst"]
    for c in qe_cols:
        df[f"{c}_log"] = np.log1p(df[c])

    qe_log_cols = [f"{c}_log" for c in qe_cols]
    df["qe_log_mean"] = df[qe_log_cols].mean(axis=1)
    df["qe_log_std"] = df[qe_log_cols].std(axis=1)
    df["qe_fast_ratio"] = (df[qe_cols] < 500).sum(axis=1) / len(qe_cols)
    df["qe_total_log"] = df[qe_log_cols].sum(axis=1)
    df["is_careless"] = ((df[qe_cols].mean(axis=1) < 500) | (df["qa_all_same"] == 1)).astype(float)

    # TP summary (Big5 style)
    tp_cols = [f"tp{i:02d}" for i in range(1, 11)]
    df["tp_missing_ratio"] = df[tp_cols].isna().sum(axis=1) / len(tp_cols)
    df["extraversion"] = df["tp01"] - df["tp06"]
    df["agreeableness"] = df["tp07"] - df["tp02"]
    df["conscientiousness"] = df["tp03"] - df["tp08"]
    df["neuroticism"] = df["tp04"] - df["tp09"]
    df["openness"] = df["tp05"] - df["tp10"]
    df["tp_mean"] = df[tp_cols].mean(axis=1)

    # WR/WF
    wr_cols = [f"wr_{i:02d}" for i in range(1, 14)]
    wf_cols = [f"wf_{i:02d}" for i in range(1, 4)]
    df["wr_sum"] = df[wr_cols].sum(axis=1)
    df["wf_sum"] = df[wf_cols].sum(axis=1)
    df["word_credibility"] = df["wr_sum"] - df["wf_sum"]
    df["vocab_high"] = (df["wr_sum"] >= 11).astype(float)

    # interaction
    df["age_edu"] = df["age_ord"] * df["education"]
    df["young_low_edu"] = df["is_young"] * df["edu_low"]
    df["young_single"] = df["is_young"] * df["is_single"]
    df["old_married"] = df["is_old"] * df["is_married"]
    df["teenager_low_edu"] = df["is_teenager"] * df["edu_low"]

    # for TE combos
    df["age_edu_cat"] = df["age_group"].astype(str) + "_" + df["education"].astype(str)
    df["age_married_cat"] = df["age_group"].astype(str) + "_" + df["married"].astype(str)
    df["age_race_cat"] = df["age_group"].astype(str) + "_" + df["race"].astype(str)
    df["age_edu_married_cat"] = df["age_group"].astype(str) + "_" + df["education"].astype(str) + "_" + df["married"].astype(str)

    return df


# ============================================================
# Target Encoding (fold 내부에서만)
# ============================================================
def target_encode(train_df, val_df, test_df, col, target_col="voted_bin", smoothing=10):
    global_mean = train_df[target_col].mean()
    agg = train_df.groupby(col)[target_col].agg(["mean", "count"])
    te = (agg["count"] * agg["mean"] + smoothing * global_mean) / (agg["count"] + smoothing)
    te_map = te.to_dict()

    return (
        train_df[col].map(te_map).fillna(global_mean).values,
        val_df[col].map(te_map).fillna(global_mean).values,
        test_df[col].map(te_map).fillna(global_mean).values,
    )


def make_te(train_df, val_df, test_df):
    out = {"train": {}, "val": {}, "test": {}}

    single_cols = [("age_group", 10), ("race", 10), ("religion", 10)]
    combo_cols  = [("age_edu_cat", 5), ("age_married_cat", 5), ("age_race_cat", 5), ("age_edu_married_cat", 3)]

    for c, sm in single_cols + combo_cols:
        tr, va, ts = target_encode(train_df, val_df, test_df, c, "voted_bin", sm)
        out["train"][f"{c}_te"] = tr
        out["val"][f"{c}_te"] = va
        out["test"][f"{c}_te"] = ts

    return out


# ============================================================
# Feature Groups (너가 원한 “정리” 그대로 코드에 반영)
# ============================================================
QA_RAW = [f"Q{c}A" for c in "abcdefghijklmnopqrst"]
QE_RAW = [f"Q{c}E_log" for c in "abcdefghijklmnopqrst"]  # log 사용
TP_RAW = [f"tp{i:02d}" for i in range(1, 11)]
WR_RAW = [f"wr_{i:02d}" for i in range(1, 14)]
WF_RAW = [f"wf_{i:02d}" for i in range(1, 4)]

RAW_FEATURES = QA_RAW + QE_RAW + TP_RAW + WR_RAW + WF_RAW + ["education", "married", "urban", "engnat", "familysize", "hand", "age_ord"]

SUMMARY_FEATURES = [
    "is_teenager", "is_young", "is_old", "edu_low", "edu_high",
    "is_single", "is_married", "is_urban", "is_english_native", "is_male",
    "qa_mean", "qa_std", "qa_range", "qa_extreme_ratio", "qa_neutral_ratio", "qa_all_same",
    "qe_log_mean", "qe_log_std", "qe_fast_ratio", "qe_total_log", "is_careless",
    "tp_missing_ratio", "tp_mean",
    "extraversion", "agreeableness", "conscientiousness", "neuroticism", "openness",
    "wr_sum", "wf_sum", "word_credibility", "vocab_high",
    "age_edu", "young_low_edu", "young_single", "old_married", "teenager_low_edu",
]

TE_FEATURES = [
    "age_group_te", "race_te", "religion_te",
    "age_edu_cat_te", "age_married_cat_te", "age_race_cat_te", "age_edu_married_cat_te",
]


# ============================================================
# Dataset
# ============================================================
class TabDataset(Dataset):
    def __init__(self, X, y=None):
        self.X = torch.tensor(X, dtype=torch.float32)
        self.y = None if y is None else torch.tensor(y, dtype=torch.float32).unsqueeze(1)

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        if self.y is None:
            return self.X[idx]
        return self.X[idx], self.y[idx]


# ============================================================
# MLP Model (Optuna로 구조 튜닝)
# ============================================================
class MLP(nn.Module):
    def __init__(self, input_dim, hidden_dims, dropout):
        super().__init__()
        layers = []
        prev = input_dim
        for h in hidden_dims:
            layers += [
                nn.Linear(prev, h),
                nn.BatchNorm1d(h),
                nn.SiLU(),
                nn.Dropout(dropout),
            ]
            prev = h
        layers += [nn.Linear(prev, 1)]
        self.net = nn.Sequential(*layers)

        # init
        for m in self.modules():
            if isinstance(m, nn.Linear):
                nn.init.xavier_uniform_(m.weight)
                if m.bias is not None:
                    nn.init.zeros_(m.bias)

    def forward(self, x):
        return self.net(x)


# ============================================================
# Train / Eval
# ============================================================
def train_one_fold(model, tr_loader, va_loader, y_va, lr, weight_decay, device):
    model.to(device)

    # imbalance 대응(너가 쓰던 방식 유지)
    pos_ratio = float(np.mean(y_va))
    pos_weight = torch.tensor([(1 - pos_ratio) / (pos_ratio + 1e-6)], device=device)
    criterion = nn.BCEWithLogitsLoss(pos_weight=pos_weight)

    opt = torch.optim.AdamW(model.parameters(), lr=lr, weight_decay=weight_decay)
    sched = torch.optim.lr_scheduler.ReduceLROnPlateau(opt, mode="max", factor=0.5, patience=3)

    best_auc, best_state, no_imp = -1, None, 0

    for epoch in range(EPOCHS):
        model.train()
        for x, y in tr_loader:
            x, y = x.to(device), y.to(device)
            opt.zero_grad()
            loss = criterion(model(x), y)
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
            opt.step()

        # val
        model.eval()
        preds = []
        with torch.no_grad():
            for x, _y in va_loader:
                x = x.to(device)
                preds.append(torch.sigmoid(model(x)).cpu().numpy())
        preds = np.concatenate(preds).ravel()
        auc = roc_auc_score(y_va, preds)
        sched.step(auc)

        if auc > best_auc + 1e-5:
            best_auc = auc
            best_state = {k: v.detach().cpu().clone() for k, v in model.state_dict().items()}
            no_imp = 0
        else:
            no_imp += 1

        if no_imp >= PATIENCE:
            break

    if best_state is not None:
        model.load_state_dict(best_state)
    return best_auc


def predict(model, loader, device):
    model.eval()
    preds = []
    with torch.no_grad():
        for x in loader:
            if isinstance(x, (tuple, list)):
                x = x[0]
            x = x.to(device)
            preds.append(torch.sigmoid(model(x)).cpu().numpy())
    return np.concatenate(preds).ravel()


# ============================================================
# Build fold matrices (feature mode로 RAW/SUM/TE 선택)
# ============================================================
def make_X(train_df, val_df, test_df, use_raw, use_summary, use_te):
    cols = []

    # RAW는 비용 대비 이득이 케이스 따라 갈려서 “옵션”으로 둠
    if use_raw:
        cols += RAW_FEATURES

    if use_summary:
        cols += SUMMARY_FEATURES

    # TE는 fold 내부에서 생성해서 붙임
    if use_te:
        te = make_te(train_df, val_df, test_df)
        for te_name in TE_FEATURES:
            train_df[te_name] = te["train"][te_name]
            val_df[te_name]   = te["val"][te_name]
            test_df[te_name]  = te["test"][te_name]
        cols += TE_FEATURES

    X_tr = train_df[cols].copy()
    X_va = val_df[cols].copy()
    X_ts = test_df[cols].copy()

    # NaN -> train median
    for c in cols:
        med = X_tr[c].median()
        if pd.isna(med):
            med = 0.0
        X_tr[c] = X_tr[c].fillna(med)
        X_va[c] = X_va[c].fillna(med)
        X_ts[c] = X_ts[c].fillna(med)

    # scale
    scaler = StandardScaler()
    X_tr = scaler.fit_transform(X_tr.values)
    X_va = scaler.transform(X_va.values)
    X_ts = scaler.transform(X_ts.values)

    return X_tr, X_va, X_ts, len(cols)


# ============================================================
# Optuna Objective (MLP only)
# ============================================================
def objective(trial: optuna.Trial, train_all: pd.DataFrame, test_all: pd.DataFrame):
    # feature mode도 같이 탐색 가능 (너가 원한 “뭐가 맞는지 테스트”를 자동화)
    use_te = trial.suggest_categorical("use_te", [True])  # TE는 고정 추천
    use_summary = trial.suggest_categorical("use_summary", [True, False])
    use_raw = trial.suggest_categorical("use_raw", [False, True])  # raw는 케이스에 따라 달라서 탐색

    # model params
    n_layers = trial.suggest_int("n_layers", 2, 4)
    base = trial.suggest_categorical("hidden_base", [128, 192, 256, 320, 384])
    shrink = trial.suggest_float("shrink", 0.55, 0.85)
    hidden_dims = []
    cur = base
    for _ in range(n_layers):
        hidden_dims.append(int(cur))
        cur = max(32, cur * shrink)

    dropout = trial.suggest_float("dropout", 0.1, 0.5)
    lr = trial.suggest_float("lr", 2e-4, 3e-3, log=True)
    weight_decay = trial.suggest_float("weight_decay", 1e-6, 3e-3, log=True)
    batch_size = trial.suggest_categorical("batch_size", [256, 512, 768])

    # CV
    y = train_all["voted_bin"].values.astype(np.float32)
    skf = StratifiedKFold(n_splits=N_FOLDS, shuffle=True, random_state=SEED)

    fold_aucs = []
    for fold, (tr_idx, va_idx) in enumerate(skf.split(train_all, y), 1):
        tr_df = train_all.iloc[tr_idx].copy().reset_index(drop=True)
        va_df = train_all.iloc[va_idx].copy().reset_index(drop=True)
        ts_df = test_all.copy()

        X_tr, X_va, _, n_feat = make_X(tr_df, va_df, ts_df, use_raw, use_summary, use_te)

        y_tr = tr_df["voted_bin"].values.astype(np.float32)
        y_va = va_df["voted_bin"].values.astype(np.float32)

        tr_ds = TabDataset(X_tr, y_tr)
        va_ds = TabDataset(X_va, y_va)

        tr_loader = DataLoader(tr_ds, batch_size=batch_size, shuffle=True, drop_last=True)
        va_loader = DataLoader(va_ds, batch_size=batch_size, shuffle=False)

        model = MLP(input_dim=n_feat, hidden_dims=hidden_dims, dropout=dropout)
        auc = train_one_fold(model, tr_loader, va_loader, y_va, lr, weight_decay, DEVICE)

        fold_aucs.append(auc)
        trial.report(float(np.mean(fold_aucs)), step=fold)

        if trial.should_prune():
            raise optuna.TrialPruned()

    mean_auc = float(np.mean(fold_aucs))

    # 너가 원한 “print 비교”
    print(f"[Trial {trial.number:03d}] AUC={mean_auc:.5f} | raw={use_raw} sum={use_summary} te={use_te} | feat={n_feat} | layers={n_layers} base={base} drop={dropout:.2f} lr={lr:.1e} wd={weight_decay:.1e}")
    return mean_auc


# ============================================================
# Train best and make submissions
# ============================================================
def train_and_predict_best(best_params, train_all, test_all):
    use_raw = best_params["use_raw"]
    use_summary = best_params["use_summary"]
    use_te = best_params["use_te"]

    # hidden dims 복원
    n_layers = int(best_params["n_layers"])
    base = int(best_params["hidden_base"])
    shrink = float(best_params["shrink"])
    hidden_dims = []
    cur = base
    for _ in range(n_layers):
        hidden_dims.append(int(cur))
        cur = max(32, cur * shrink)

    dropout = float(best_params["dropout"])
    lr = float(best_params["lr"])
    weight_decay = float(best_params["weight_decay"])
    batch_size = int(best_params["batch_size"])

    y = train_all["voted_bin"].values.astype(np.float32)
    skf = StratifiedKFold(n_splits=N_FOLDS, shuffle=True, random_state=SEED)

    oof = np.zeros(len(train_all), dtype=np.float32)
    test_pred = np.zeros(len(test_all), dtype=np.float32)

    fold_aucs = []
    for fold, (tr_idx, va_idx) in enumerate(skf.split(train_all, y), 1):
        tr_df = train_all.iloc[tr_idx].copy().reset_index(drop=True)
        va_df = train_all.iloc[va_idx].copy().reset_index(drop=True)
        ts_df = test_all.copy()

        X_tr, X_va, X_ts, n_feat = make_X(tr_df, va_df, ts_df, use_raw, use_summary, use_te)

        y_tr = tr_df["voted_bin"].values.astype(np.float32)
        y_va = va_df["voted_bin"].values.astype(np.float32)

        tr_ds = TabDataset(X_tr, y_tr)
        va_ds = TabDataset(X_va, y_va)
        ts_ds = TabDataset(X_ts)

        tr_loader = DataLoader(tr_ds, batch_size=batch_size, shuffle=True, drop_last=True)
        va_loader = DataLoader(va_ds, batch_size=batch_size, shuffle=False)
        ts_loader = DataLoader(ts_ds, batch_size=batch_size, shuffle=False)

        model = MLP(input_dim=n_feat, hidden_dims=hidden_dims, dropout=dropout)
        auc = train_one_fold(model, tr_loader, va_loader, y_va, lr, weight_decay, DEVICE)
        fold_aucs.append(auc)

        oof[va_idx] = predict(model, va_loader, DEVICE)
        test_pred += predict(model, ts_loader, DEVICE) / N_FOLDS

        print(f"[BEST][Fold {fold}] AUC={auc:.5f} | n_feat={n_feat}")

    oof_auc = roc_auc_score(train_all["voted_bin"], oof)
    print(f"\n✅ BEST OOF AUC: {oof_auc:.5f} | fold mean: {np.mean(fold_aucs):.5f}")

    return oof_auc, test_pred, fold_aucs


def auto_name(prefix, oof_auc, params):
    t = time.strftime("%m%d_%H%M%S")
    mode = f"te{int(params['use_te'])}_sum{int(params['use_summary'])}_raw{int(params['use_raw'])}"
    return f"{prefix}_{mode}_auc{oof_auc:.5f}_{t}"


def main():
    set_seed(SEED)

    train_clean = clean_data(train_raw)
    test_clean  = clean_data(test_raw)

    train_fe = build_features(train_clean)
    test_fe  = build_features(test_clean)

    sampler = optuna.samplers.TPESampler(seed=SEED)
    pruner  = optuna.pruners.MedianPruner(n_warmup_steps=2)

    study = optuna.create_study(
        direction="maximize",
        sampler=sampler,
        pruner=pruner,
        study_name=OPTUNA_STUDY_NAME,
        storage=OPTUNA_STORAGE,
        load_if_exists=True,
    )

    print("\n" + "="*80)
    print(f"🚀 OPTUNA START | trials={N_TRIALS} | timeout={TIMEOUT_SEC}")
    print("="*80)

    study.optimize(lambda tr: objective(tr, train_fe, test_fe), n_trials=N_TRIALS, timeout=TIMEOUT_SEC)

    print("\n" + "="*80)
    print("🏁 OPTUNA DONE")
    print("="*80)
    print("Best AUC:", study.best_value)
    print("Best params:\n", json.dumps(study.best_params, indent=2))

    # train best and predict
    best_auc, test_pred, fold_aucs = train_and_predict_best(study.best_params, train_fe, test_fe)

    # submission auto naming
    base = auto_name("submission_mlp_optuna", best_auc, study.best_params)

    sub = pd.DataFrame({
        "index": test_raw["index"] if "index" in test_raw.columns else np.arange(len(test_raw)),
        "voted": test_pred
    })
    sub_path = f"{base}.csv"
    sub.to_csv(sub_path, index=False)

    # save params too
    with open(f"{base}_params.json", "w") as f:
        json.dump({"best_auc": best_auc, "fold_aucs": fold_aucs, "params": study.best_params}, f, indent=2)

    print(f"\n💾 Saved: {sub_path}")
    print(f"💾 Saved: {base}_params.json")
    print(f"   pred range: [{test_pred.min():.4f}, {test_pred.max():.4f}]")


if __name__ == "__main__":
    main()


🖥️ Device: cpu
📂 Train path: ../../data/raw/train.csv
📂 Test  path: ../../data/raw/test_x.csv
Train: (45532, 79), Test: (11383, 77)


[I 2026-01-30 14:43:44,527] A new study created in memory with name: MLP_TE



🚀 OPTUNA START | trials=50 | timeout=None


[I 2026-01-30 14:44:37,886] Trial 0 finished with value: 0.7590648796261102 and parameters: {'use_te': True, 'use_summary': False, 'use_raw': False, 'n_layers': 2, 'hidden_base': 256, 'shrink': 0.5561753482887408, 'dropout': 0.4879639408647978, 'lr': 0.0019057174434080474, 'weight_decay': 5.474303040596577e-06, 'batch_size': 768}. Best is trial 0 with value: 0.7590648796261102.


[Trial 000] AUC=0.75906 | raw=False sum=False te=True | feat=7 | layers=2 base=256 drop=0.49 lr=1.9e-03 wd=5.5e-06


...

[Trial 047] AUC=0.76725 | raw=False sum=True te=True | feat=44 | layers=4 base=320 drop=0.42 lr=1.3e-03 wd=3.8e-04


[I 2026-01-30 16:16:36,505] Trial 48 finished with value: 0.7674712415498529 and parameters: {'use_te': True, 'use_summary': False, 'use_raw': True, 'n_layers': 3, 'hidden_base': 192, 'shrink': 0.717267986566778, 'dropout': 0.3767220330048511, 'lr': 0.002078654917538259, 'weight_decay': 3.4473085526509026e-06, 'batch_size': 512}. Best is trial 27 with value: 0.7692024957969453.


[Trial 048] AUC=0.76747 | raw=True sum=False te=True | feat=80 | layers=3 base=192 drop=0.38 lr=2.1e-03 wd=3.4e-06


[I 2026-01-30 16:18:55,063] Trial 49 finished with value: 0.7659989530261372 and parameters: {'use_te': True, 'use_summary': True, 'use_raw': True, 'n_layers': 3, 'hidden_base': 256, 'shrink': 0.8458404250813222, 'dropout': 0.19235459996755194, 'lr': 0.00027467204985373025, 'weight_decay': 0.0009231193194598854, 'batch_size': 512}. Best is trial 27 with value: 0.7692024957969453.


[Trial 049] AUC=0.76600 | raw=True sum=True te=True | feat=117 | layers=3 base=256 drop=0.19 lr=2.7e-04 wd=9.2e-04

🏁 OPTUNA DONE
Best AUC: 0.7692024957969453
Best params:
 {
  "use_te": true,
  "use_summary": true,
  "use_raw": true,
  "n_layers": 3,
  "hidden_base": 256,
  "shrink": 0.8212081026195875,
  "dropout": 0.400484176222794,
  "lr": 0.0014692944373311206,
  "weight_decay": 0.001847088696674732,
  "batch_size": 512
}
[BEST][Fold 1] AUC=0.78046 | n_feat=117
[BEST][Fold 2] AUC=0.76859 | n_feat=117
[BEST][Fold 3] AUC=0.76236 | n_feat=117
[BEST][Fold 4] AUC=0.76231 | n_feat=117
[BEST][Fold 5] AUC=0.76968 | n_feat=117

✅ BEST OOF AUC: 0.76833 | fold mean: 0.76868

💾 Saved: submission_mlp_optuna_te1_sum1_raw1_auc0.76833_0130_162044.csv
💾 Saved: submission_mlp_optuna_te1_sum1_raw1_auc0.76833_0130_162044_params.json
   pred range: [0.0935, 0.9987]
